Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resampling import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.indicators import filter_in_market, screen_one
from os.path import exists, join

import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=10, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input'):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param input_log_name: the directory where to take the logs from.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         raw=False)

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def screen(self, dataset):
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'base_volume': 'volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            assets = filter_in_market(screen_one, dataset)
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'volume': 'base_volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def resample_from_raw(self, df):
        df = df[['symbol', 'close', 'rolling_base_volume', 'rolling_quote_volume']]
        df['base_volume'] = df['rolling_base_volume'].copy()
        df['quote_volume'] = df['rolling_quote_volume'].copy()
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume', 
                                    'base_volume', 'quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'base_volume': 'max', 'quote_volume': 'max', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=('pair', 'symbol'))
        df = df.rename(columns={'close_first': 'open', 'close_max': 'high', 
                                'close_min': 'low', 'close_last': 'close', 
                                'base_volume_max': 'base_volume', 
                                'quote_volume_max': 'quote_volume', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['base_volume'] = df['base_volume'].fillna(method='pad')
        df['base_volume'].iloc[0] = 0
        df['quote_volume'] = df['quote_volume'].fillna(method='pad')
        df['quote_volume'].iloc[0] = 0
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        dataset = self.get_from_file(log_name=self.input_log_name, 
                                     from_raw=not self.load_from_ohlcv)
        if not self.load_from_ohlcv:
            dataset = self.resample_from_raw(dataset)
        return dataset.tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

crypto_logger_output_1min = Crypto_logger_output(delay=44, 
                                                 interval_input='30s', 
                                                 interval='1min', 
                                                 buffer_size=1500, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
import pandas as pd
input_log_name = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(input_log_name, header=0, index_col=0)
df.index = pd.DatetimeIndex(df.index)
df = df.sort_index(axis='index')
#df = crypto_logger_output_1min.resample_from_raw(df)
df

In [ ]:
import pandas as pd
#crypto_output_log_1min = crypto_logger_output_1min.log_name
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min = df_1min.sort_index(axis='index')
df_1min['BTCUSDT'].tail(20)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [ ]:
def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iloc[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iloc[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iloc[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iloc[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iloc[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iloc[-2]) or \
            (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

In [ ]:
filter_in_market(get_positive_phase_trigger, df)

In [ ]:
get_positive_phase_trigger(df['BTCUSDT'])

In [ ]:
from tqdm import tqdm

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tqdm(tickers_list, unit=' ticker') if f(dataset[ticker])], dtype='str')

def get_heikin_ashi_trigger(ticker):
    def get_trend_strength_entry_trigger(heikin_ashi_dataset):
        ADX = heikin_ashi_dataset.ta.adx(talib=True)
        return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

    def get_not_negative_rebound_trigger(heikin_ashi_dataset):
        CCI = heikin_ashi_dataset.ta.cci(talib=True)
        return CCI.iloc[-1] < 100

    def get_positive_choppiness_trigger(heikin_ashi_dataset):
        CHOP = heikin_ashi_dataset.ta.chop(talib=True)
        return CHOP.iloc[-1] < 38.2

    def get_buy_trigger(heikin_ashi_dataset):
        return get_not_negative_rebound_trigger(heikin_ashi_dataset) and \
               (get_positive_choppiness_trigger(heikin_ashi_dataset) or \
                get_trend_strength_entry_trigger(heikin_ashi_dataset))

    heikin_ashi = ticker.ta.ha(talib=True)
    heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                      'HA_high': 'high', 
                                                      'HA_low': 'low', 
                                                      'HA_close': 'close'})
    try:
        trigger = get_buy_trigger(heikin_ashi_dataset)
    except:
        trigger = False
    return trigger

tickers_list = filter_in_market(get_heikin_ashi_trigger, dataset)
tickers_list

In [ ]:
heikin_ashi = dataset['BTCUSDT'].ta.ha(talib=True)
heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                  'HA_high': 'high', 
                                                  'HA_low': 'low', 
                                                  'HA_close': 'close'})
ADX = heikin_ashi_dataset.ta.adx(talib=True)
ADX[['DMP_14', 'DMN_14']]

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [4]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [23]:
from cryptocurrency.conversion import convert_price, get_base_asset_from_pair, get_quote_asset_from_pair
from pandas import concat, DataFrame

def get_conversion_table(client, exchange_info, as_pair=False):
    """
    Fetches and prepares data used to calculate prices, volumes and other stats.
    :param client: object from python-binance useful for calling client.get_ticker().
    :param exchange_info: Pre-calculated exchange information on all tickers.
    :return: pd.DataFrame containing all preprocessed conversion table info.
    :column is_shorted: is the symbol made from inversion.
    :column symbol: concatenated string made from base_asset and quote_asset.
    :column shorted_symbol: symbol with inverted base_asset and quote_asset.
    :column base_asset: asset on the left.
    :column quote_asset: asset on the right.
    :column price_change: (close - open).
    :column price_change_percent: ((close - open) / open).
    :column weighted_average_price: weighted average price.
    :column close_shifted: close price of the previous day.
    :column open: open price of the day.
    :column high: high price of the day.
    :column low: low price of the day.
    :column close: close price of the day.
    :column last_volume: volume of the last price update.
    :column USDT_bid_price: USDT-converted bid price.
    :column USDT_ask_price: USDT-converted ask price.
    :column USDT_bid_volume: USDT-converted bid volume.
    :column USDT_ask_volume: USDT-converted ask volume.
    :column bid_price: price of the bid.
    :column bid_volume: volume of the bid at bid_price.
    :column ask_price: price of the ask.
    :column ask_volume: volume of the ask at ask_price.
    :column rolling_base_volume: rolling_base_volume given by the API.
    :column rolling_quote_volume: rolling_quote_volume given by the API.
    :column open_time: close_time minus 24 hours.
    :column close_time: time from epoch in milliseconds of the last price update.
    :column first_ID: transaction ID from 1 day ago.
    :column last_ID: latest transaction ID.
    :column count: value calculated by subtracting first_ID from last_ID.
    :column USDT_price: value calculated by subtracting firstId from lastId.
    :column rolling_USDT_base_volume: USDT-converted rolling_base_volume.
    :column rolling_USDT_quote_volume: USDT-converted rolling_quote_volume.
    :column rolling_traded_volume: sum by base_asset of all USDT-converted volumes.
    :column importance: rolling_USDT_base_volume divided by rolling_traded_volume.
    :column traded_price: sum by base_asset of all (close prices times importance).
    :column traded_bid_price: sum by base_asset of all (bid prices times importance).
    :column traded_ask_price: sum by base_asset of all (ask prices times importance).
    :column bid_ask_change_percent: ((ask_price - bid_price) / ask_price) * 100).
    :column bid_ask_volume_percent: ((bid_volume / (bid_volume + ask_volume)) * 100).
    :column traded_bid_ask_change_percent: ((traded_ask_price - traded_bid_price) / traded_ask_price) * 100).
    :column traded_bid_ask_volume_percent: ((traded_bid_volume / (traded_bid_volume + traded_ask_volume)) * 100).
    """
    conversion_table = DataFrame(client.get_ticker())

    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])]
    conversion_table['base_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
    conversion_table['quote_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=exchange_info))

    conversion_table = \
        conversion_table.rename(columns={'openPrice': 'open', 'highPrice': 'high', 'lowPrice': 'low', 
                                         'lastPrice': 'close', 'volume': 'rolling_base_volume', 
                                         'quoteVolume': 'rolling_quote_volume', 'lastQty': 'last_volume', 
                                         'bidPrice': 'bid_price', 'askPrice': 'ask_price', 
                                         'bidQty': 'bid_volume', 'askQty': 'ask_volume', 
                                         'firstId': 'first_ID', 'lastId': 'last_ID', 
                                         'openTime': 'open_time', 'closeTime': 'close_time', 
                                         'prevClosePrice': 'close_shifted', 
                                         'weightedAvgPrice': 'weighted_average_price', 
                                         'priceChange': 'price_change', 
                                         'priceChangePercent': 'price_change_percent'})
    conversion_table[['price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                      'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 'rolling_quote_volume', 
                      'count']] = \
        conversion_table[['price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                          'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 'rolling_quote_volume', 
                          'count']].astype(float)

    conversion_table['rolling_base_quote_volume'] = \
        conversion_table['rolling_quote_volume'] / conversion_table['close']
    conversion_table['USDT_price'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), axis='columns')
    conversion_table['rolling_USDT_base_volume'] = \
        conversion_table['rolling_base_volume'] * conversion_table['USDT_price']
    conversion_table['rolling_USDT_quote_volume'] = \
        conversion_table['rolling_base_quote_volume'] * conversion_table['USDT_price']

    conversion_table['USDT_bid_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['bid_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_ask_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['ask_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_bid_volume'] = \
        conversion_table['bid_volume'] * conversion_table['USDT_bid_price']
    conversion_table['USDT_ask_volume'] = \
        conversion_table['ask_volume'] * conversion_table['USDT_ask_price']

    conversion_table['is_shorted'] = False

    conversion_table_swapped = conversion_table.copy()
    conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                     'weighted_average_price', 'close_shifted', 'close', 
                                     'last_volume', 'ask_price', 'ask_volume', 'bid_price', 
                                     'bid_volume', 'open', 'high', 'low', 'rolling_quote_volume', 
                                     'rolling_base_volume', 'open_time', 'close_time', 'first_ID', 
                                     'last_ID', 'count', 'quote_asset', 'base_asset', 
                                     'rolling_base_quote_volume', 'USDT_price', 
                                     'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                     'USDT_ask_price', 'USDT_bid_price', 'USDT_ask_volume', 
                                     'USDT_bid_volume', 'is_shorted']] = \
        conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 'weighted_average_price', 
                                         'close_shifted', 'close', 'last_volume', 'bid_price', 'bid_volume', 
                                         'ask_price', 'ask_volume', 'open', 'high', 'low', 'rolling_base_volume', 
                                         'rolling_quote_volume', 'open_time', 'close_time', 'first_ID', 'last_ID', 
                                         'count', 'base_asset', 'quote_asset', 'rolling_base_quote_volume', 
                                         'USDT_price', 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                         'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 'USDT_ask_volume', 
                                         'is_shorted']].values
    conversion_table_swapped['symbol'] = \
        conversion_table_swapped['base_asset'] + conversion_table_swapped['quote_asset']
    conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 'ask_price', 
                                     'USDT_price', 'USDT_bid_price', 'USDT_ask_price']] = \
        1 / conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 
                                             'bid_price', 'ask_price', 'USDT_price', 
                                             'USDT_bid_price', 'USDT_ask_price']].astype(float)
    conversion_table_swapped['is_shorted'] = True

    conversion_table = concat([conversion_table, conversion_table_swapped], join='outer', axis='index')

    traded_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_volume = traded_volume['rolling_USDT_base_volume']
    conversion_table['rolling_traded_volume'] = \
        conversion_table.apply(lambda x: traded_volume.loc[x['base_asset']], axis='columns')
    traded_bid_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_bid_volume = traded_bid_volume['USDT_bid_volume']
    conversion_table['traded_bid_volume'] = \
        conversion_table.apply(lambda x: traded_bid_volume.loc[x['base_asset']], axis='columns')
    traded_ask_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_ask_volume = traded_ask_volume['USDT_ask_volume']
    conversion_table['traded_ask_volume'] = \
        conversion_table.apply(lambda x: traded_ask_volume.loc[x['base_asset']], axis='columns')

    conversion_table['importance'] = \
        conversion_table['rolling_USDT_base_volume'] / conversion_table['rolling_traded_volume']

    conversion_table['importance_weighted_price'] = \
        conversion_table['USDT_price'] * conversion_table['importance']
    conversion_table['importance_weighted_bid_price'] = \
        conversion_table['USDT_bid_price'] * conversion_table['importance']
    conversion_table['importance_weighted_ask_price'] = \
        conversion_table['USDT_ask_price'] * conversion_table['importance']

    importance_weighted_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_price = importance_weighted_price['importance_weighted_price']
    conversion_table['traded_price'] = \
        conversion_table.apply(lambda x: importance_weighted_price.loc[x['base_asset']], axis='columns')
    importance_weighted_bid_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_bid_price = importance_weighted_bid_price['importance_weighted_bid_price']
    conversion_table['traded_bid_price'] = \
        conversion_table.apply(lambda x: importance_weighted_bid_price.loc[x['base_asset']], axis='columns')
    importance_weighted_ask_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_ask_price = importance_weighted_ask_price['importance_weighted_ask_price']
    conversion_table['traded_ask_price'] = \
        conversion_table.apply(lambda x: importance_weighted_ask_price.loc[x['base_asset']], axis='columns')

    conversion_table['bid_ask_change_percent'] = \
        ((conversion_table['ask_price'] - conversion_table['bid_price']) / conversion_table['ask_price'])
    conversion_table['bid_ask_volume_percent'] = \
        (conversion_table['bid_volume'] / (conversion_table['bid_volume'] + conversion_table['ask_volume']))
    conversion_table[['bid_ask_change_percent', 'bid_ask_volume_percent']] *= 100
    conversion_table['traded_bid_ask_change_percent'] = \
        ((conversion_table['traded_ask_price'] - conversion_table['traded_bid_price']) / \
         conversion_table['traded_ask_price'])
    conversion_table['traded_bid_ask_volume_percent'] = \
        (conversion_table['traded_bid_volume'] / (conversion_table['traded_bid_volume'] + \
                                                  conversion_table['traded_ask_volume']))
    conversion_table[['traded_bid_ask_change_percent', 'traded_bid_ask_volume_percent']] *= 100

    conversion_table = conversion_table[~conversion_table['is_shorted']]
    if as_pair:
        conversion_table = \
            conversion_table[['symbol', 'base_asset', 'quote_asset', 'is_shorted', 'price_change_percent', 
                              'weighted_average_price', 'open', 'high', 'low', 'close', 'close_shifted', 
                              'last_volume', 'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 
                              'close_time', 'last_ID', 'count', 'rolling_base_volume', 'rolling_quote_volume', 
                              'importance', 'USDT_price', 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                              'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 'USDT_ask_volume', 
                              'rolling_traded_volume', 'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                              'traded_bid_price', 'traded_ask_price', 'bid_ask_change_percent', 
                              'bid_ask_volume_percent', 'traded_bid_ask_change_percent', 
                              'traded_bid_ask_volume_percent']]
    else:
        conversion_table = \
            conversion_table[['base_asset', 'price_change_percent', 'close_time', 'last_ID', 'count', 
                              'rolling_traded_volume', 'traded_bid_volume', 'traded_ask_volume', 
                              'traded_price', 'traded_bid_price', 'traded_ask_price', 
                              'traded_bid_ask_change_percent', 'traded_bid_ask_volume_percent']]
        conversion_table['rolling_quote_volume'] = conversion_table['rolling_traded_volume'].copy()
        conversion_table = \
            conversion_table.rename(columns={'rolling_traded_volume': 'rolling_base_volume', 
                                             'traded_bid_volume': 'bid_volume', 
                                             'traded_ask_volume': 'ask_volume', 
                                             'traded_price': 'close', 
                                             'traded_bid_price': 'bid_price', 
                                             'traded_ask_price': 'ask_price', 
                                             'traded_bid_ask_change_percent': 'bid_ask_change_percent', 
                                             'traded_bid_ask_volume_percent': 'bid_ask_volume_percent'})
        conversion_table['symbol'] = conversion_table['base_asset'].copy()
        conversion_table['quote_asset'] = conversion_table['base_asset'].copy()
        df = conversion_table.groupby(by=['base_asset']).agg({'price_change_percent': 'max', 
                                                              'close_time': 'max', 
                                                              'last_ID': 'sum', 
                                                              'count': 'sum'})
        conversion_table.loc[:, ['price_change_percent', 'close_time', 'last_ID', 'count']] = \
            conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
        conversion_table = conversion_table.drop_duplicates(subset=['base_asset'], keep='first')
        conversion_table = conversion_table.reset_index(drop=True)

    conversion_table = conversion_table.sort_values(by='close_time')
    conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=False)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)
conversion_table

,base_asset,price_change_percent,close_time,last_ID,count,rolling_base_volume,bid_volume,ask_volume,close,bid_price,ask_price,bid_ask_change_percent,bid_ask_volume_percent,rolling_quote_volume,symbol,quote_asset
0,BDOT,-0.285,1.667106e+12,2.307900e+05,425.0,8.757419e+04,7.100469e+00,2.311503e+01,4.860994,3.575261e+00,3.578178e+00,0.081511,23.499427,8.757419e+04,BDOT,BDOT
1,MITH,-1.916,1.667107e+12,4.910191e+07,4889.0,6.594729e+05,4.511217e+00,4.972696e+00,0.019960,3.207362e-04,3.213806e-04,0.200514,47.567045,6.594729e+05,MITH,MITH
2,OAX,-3.405,1.667107e+12,1.191016e+07,8330.0,1.113342e+06,1.184900e-02,4.167843e-03,0.159048,1.213540e-06,1.218311e-06,0.391645,73.978379,1.113342e+06,OAX,OAX
3,IQ,0.000,1.667107e+12,1.066433e+07,1390.0,2.354599e+05,1.585376e-01,1.377018e-01,0.004509,2.028891e-05,2.033400e-05,0.221729,53.516703,2.354599e+05,IQ,IQ
4,ALPACA,-0.750,1.667107e+12,1.335498e+07,5271.0,6.432286e+05,5.007895e+01,8.133470e+02,0.291000,7.495017e-02,7.498665e-02,0.048661,5.800028,6.432286e+05,ALPACA,ALPACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,AXS,2.817,1.667107e+12,1.572982e+08,51182.0,1.407353e+07,3.958750e+05,1.563241e+05,9.490000,1.166394e+02,1.167763e+02,0.117271,71.690628,1.407353e+07,AXS,AXS
350,BTC,1.217,1.667107e+12,2.822769e+09,8501614.0,7.956059e+09,1.924636e+09,1.707090e+11,19328.176881,8.721168e+08,8.724785e+08,0.041448,1.114867,7.956059e+09,BTC,BTC
351,ELF,-0.301,1.667107e+12,3.291902e+07,5835.0,1.426570e+06,4.026001e+01,6.548995e+01,0.132300,1.016614e-02,1.017483e-02,0.085465,38.070944,1.426570e+06,ELF,ELF
352,APT,-6.496,1.667107e+12,7.661065e+06,537410.0,2.145545e+08,4.898713e+05,1.498775e+05,7.811400,9.880354e+01,9.888308e+01,0.080431,76.572447,2.145545e+08,APT,APT


In [22]:
conversion_table.drop_duplicates(subset=['base_asset'], keep='first')

,base_asset,price_change_percent,close_time,last_ID,count,rolling_base_volume,bid_volume,ask_volume,close,bid_price,ask_price,bid_ask_change_percent,bid_ask_volume_percent,rolling_quote_volume,symbol,quote_asset
0,GHST,-0.528,1.667106e+12,5997081.0,1888.0,2.080765e+05,1.130621e+03,3.434330e+03,1.130000,1.275577e+00,1.277286e+00,0.133777,24.767436,2.080765e+05,GHST,GHST
2,IQ,0.222,1.667106e+12,10664332.0,1389.0,2.359624e+05,1.588891e-01,1.303926e+00,0.004519,2.033390e-05,2.042427e-05,0.442478,10.861868,2.359624e+05,IQ,IQ
3,BDOT,-0.285,1.667106e+12,230790.0,425.0,8.757419e+04,7.100469e+00,2.311503e+01,4.860994,3.575261e+00,3.578178e+00,0.081511,23.499427,8.757419e+04,BDOT,BDOT
4,TRIBE,2.564,1.667107e+12,6161744.0,2468.0,3.961833e+05,5.503398e+01,2.512377e+01,0.252000,6.340320e-02,6.355398e-02,0.237254,68.657091,3.961833e+05,TRIBE,TRIBE
6,BIFI,0.924,1.667107e+12,5688773.0,7582.0,9.661255e+05,4.498088e+04,1.302833e+05,371.100000,1.375445e+05,1.377744e+05,0.166882,25.664612,9.661255e+05,BIFI,BIFI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,TFUEL,12.564,1.667107e+12,58710547.0,98416.0,2.236292e+07,7.256116e+01,2.211174e+02,0.066300,3.391888e-03,3.402136e-03,0.301205,24.707681,2.236292e+07,TFUEL,TFUEL
1396,BUSD,0.774,1.667107e+12,401401628.0,641415.0,6.263756e+09,3.027798e+08,2.398196e+09,3176.366290,3.335708e+08,3.350369e+08,0.437585,11.210016,6.263756e+09,BUSD,BUSD
1406,SHIB,15.120,1.667107e+12,495461741.0,2146603.0,9.565667e+08,1.180769e+00,4.223632e-01,0.000013,6.702543e-10,6.706500e-10,0.059001,73.653882,9.565667e+08,SHIB,SHIB
1407,WAVES,15.138,1.667107e+12,141300637.0,194235.0,4.882543e+07,2.197963e+05,1.176958e+05,3.816000,3.112481e+01,3.115979e+01,0.112253,65.126352,4.882543e+07,WAVES,WAVES


In [ ]:
conversion_table[conversion_table['symbol'] == 'NEBLBUSD'][['bid_ask_volume_percent', 'traded_bid_ask_volume_percent']]

In [ ]:
symbol = 'NEBLBUSD'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [ ]:
import pandas as pd
crypto_output_log_30s = 'crypto_logs/crypto_output_log_30s.txt'
df_30s = pd.read_csv(crypto_output_log_30s, header=[0, 1], index_col=0)
df_30s.index = pd.DatetimeIndex(df_30s.index)
df_30s

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
df_15s[symbol]

In [ ]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

In [ ]:
df_1min[symbol].tail(20)

In [ ]:
import pandas as pd
crypto_output_log_5min = 'crypto_logs/crypto_output_log_5min.txt'
df_5min = pd.read_csv(crypto_output_log_5min, header=[0, 1], index_col=0)
df_5min.index = pd.DatetimeIndex(df_5min.index)
df_5min

In [ ]:
df_5min[symbol].tail(20)

In [ ]:
df = df_1min.copy()
df

In [1]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

/opt/conda/envs/rltrader/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,AAVEBNB,AAVEBTC,AAVEBUSD,AAVEETH,AAVEUSDT,ACABTC,ACABUSD,...,ZILBTC,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
pair,open,open,open,open,open,open,open,open,open,open,...,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume,quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-28 08:00:00,0.00003,0.605,0.606,0.2829,0.004032,81.3,0.0543,81.3,0.000008,0.1624,...,0.000000,0.000000,0.000000,0.0,0.0000,2990.843143,0.000000,0.0,0.0,0.0000
2022-10-28 08:01:00,0.00003,0.605,0.606,0.2828,0.004031,81.4,0.0543,81.3,0.000008,0.1624,...,0.000000,120.802884,0.000000,0.0,0.0000,100.065240,0.000000,0.0,0.0,28.3630
2022-10-28 08:02:00,0.00003,0.605,0.606,0.2828,0.004031,81.4,0.0543,81.3,0.000008,0.1624,...,0.000000,60.588000,0.000000,0.0,0.0000,1629.708483,0.000000,0.0,0.0,25.5918
2022-10-28 08:03:00,0.00003,0.606,0.606,0.2828,0.004032,81.4,0.0543,81.3,0.000008,0.1624,...,0.000568,0.000000,0.132687,0.0,0.0000,224.937500,0.014867,0.0,0.0,0.0000
2022-10-28 08:04:00,0.00003,0.607,0.607,0.2828,0.004033,81.4,0.0543,81.4,0.000008,0.1624,...,0.000000,0.000000,0.000000,0.0,0.0000,1289.628902,0.009778,0.0,0.0,624.1560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 15:58:00,0.00003,0.617,0.617,0.2810,0.004080,84.0,0.0540,84.1,0.000008,0.1666,...,0.000000,234.105846,0.000000,0.0,3386.5104,8052.255813,0.000000,0.0,0.0,384.9625
2022-10-28 15:59:00,0.00003,0.617,0.618,0.2810,0.004080,84.0,0.0540,84.1,0.000008,0.1666,...,0.000000,0.000000,0.000000,0.0,0.0000,4088.660565,0.000000,0.0,0.0,0.0000


In [2]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1431/1431 [00:05<00:00, 256.02 pair conversion/s]
1431 named pair [00:24, 58.15 named pair/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1431/1431 [00:05<00:00, 243.70 pair conversion/s]
1431 named pair [00:25, 56.13 named pair/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1431/1431 [00:00<00:00, 4749.20 pair/s]
1431 named pair [00:38, 36.71 named pair/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1431/1431 [00:00<00:00, 4644.71 pair/s]
1431 named pair [00:34, 42.00 named pair/s]
  0%|                                                                                                                                       | 0/365 

asset                1INCH                                                \
pair                  open   high    low  close base_volume quote_volume   
date                                                                       
2022-10-28 08:00:00  0.606  0.606  0.606  0.606    222.8262   222.765640   
2022-10-28 08:01:00  0.606  0.606  0.606  0.606     99.9900    99.990000   
2022-10-28 08:02:00  0.606  0.606  0.606  0.606   2862.1380  2862.137839   
2022-10-28 08:03:00  0.606  0.607  0.606  0.607   3208.4806  3206.814000   
2022-10-28 08:04:00  0.607  0.608  0.607  0.608   2467.1424  2465.988689   
...                    ...    ...    ...    ...         ...          ...   
2022-10-28 15:58:00  0.617  0.618  0.617  0.618   3658.1274  3657.395600   
2022-10-28 15:59:00  0.618  0.619  0.618  0.618   5156.6538  5159.271300   
2022-10-28 16:00:00  0.619  0.619  0.618  0.618   1135.2660  1137.078900   
2022-10-28 16:01:00  0.619  0.619  0.619  0.619    250.1379   250.137900   
2022-10-28 16:02:00  0.619  0.619  0.619  0.619      0.0000     0.000000   

asset                AAVE                    ...      ZIL           \
pair                 open  high   low close  ...      low    close   
date                                         ...                     
2022-10-28 08:00:00  81.3  81.4  81.3  81.3  ...  0.02969  0.02969   
2022-10-28 08:01:00  81.3  81.3  81.3  81.3  ...  0.02970  0.02970   
2022-10-28 08:02:00  81.3  81.3  81.3  81.3  ...  0.02969  0.02969   
2022-10-28 08:03:00  81.3  81.4  81.3  81.4  ...  0.02968  0.02969   
2022-10-28 08:04:00  81.4  81.4  81.4  81.4  ...  0.02969  0.02970   
...                   ...   ...   ...   ...  ...      ...      ...   
2022-10-28 15:58:00  84.1  84.1  84.1  84.1  ...  0.03050  0.03052   
2022-10-28 15:59:00  84.1  84.1  84.1  84.1  ...  0.03053  0.03054   
2022-10-28 16:00:00  84.1  84.1  84.1  84.1  ...  0.03054  0.03056   
2022-10-28 16:01:00  84.1  84.1  84.1  84.1  ...  0.03056  0.03056   
2022-10-28 16:02:00  84.1  84.1  84.1  84.1  ...  0.03056  0.03056   

asset                                               ZRX                  \
pair                  base_volume  quote_volume    open    high     low   
date                                                                      
2022-10-28 08:00:00   2989.880977   2990.843143  0.2511  0.2511  0.2511   
2022-10-28 08:01:00    220.786830    220.786830  0.2510  0.2510  0.2510   
2022-10-28 08:02:00   1690.272483   1690.276083  0.2509  0.2509  0.2509   
2022-10-28 08:03:00    435.210865    435.202080  0.2509  0.2509  0.2509   
2022-10-28 08:04:00   1290.043260   1289.628902  0.2511  0.2511  0.2510   
...                           ...           ...     ...     ...     ...   
2022-10-28 15:58:00   8462.768720   8466.739141  0.2575  0.2575  0.2575   
2022-10-28 15:59:00   4088.710470   4088.660565  0.2575  0.2575  0.2575   
2022-10-28 16:00:00  14085.406544  14088.574210  0.2575  0.2575  0.2575   
2022-10-28 16:01:00      9.999232      9.999232  0.2575  0.2575  0.2575   
2022-10-28 16:02:00      0.000000      0.000000  0.2575  0.2575  0.2575   

asset                                                 
pair                  close base_volume quote_volume  
date                                                  
2022-10-28 08:00:00  0.2511      0.0000     0.000000  
2022-10-28 08:01:00  0.2510     28.3630    28.363000  
2022-10-28 08:02:00  0.2509     25.5918    25.591800  
2022-10-28 08:03:00  0.2509    299.8255   299.603015  
2022-10-28 08:04:00  0.2511    821.5992   821.520600  
...                     ...         ...          ...  
2022-10-28 15:58:00  0.2575    384.9625   384.962500  
2022-10-28 15:59:00  0.2575      0.0000     0.000000  
2022-10-28 16:00:00  0.2575      0.0000     0.000000  
2022-10-28 16:01:00  0.2575      0.0000     0.000000  
2022-10-28 16:02:00  0.2575      0.0000     0.000000  

[483 rows x 2124 columns]

In [3]:
new_conversion_table['ETH']

pair,open,high,low,close,base_volume,quote_volume
date,,,,,,
2022-10-28 08:00:00,1498.19,1498.45,1497.09,1497.10,8.870000e+05,8.874185e+05
2022-10-28 08:01:00,1497.09,1498.31,1496.92,1498.04,5.244474e+05,5.242223e+05
2022-10-28 08:02:00,1498.04,1498.04,1496.73,1496.94,3.873023e+05,3.873735e+05
2022-10-28 08:03:00,1496.94,1499.15,1496.94,1498.75,1.392143e+06,1.391288e+06
2022-10-28 08:04:00,1498.75,1499.13,1498.40,1498.85,2.546707e+05,2.546397e+05
...,...,...,...,...,...,...
2022-10-28 15:58:00,1554.77,1557.06,1554.76,1556.59,5.966726e+05,5.964818e+05
2022-10-28 15:59:00,1556.58,1557.95,1556.58,1557.36,7.994883e+05,7.994573e+05
2022-10-28 16:00:00,1557.37,1558.93,1557.16,1558.21,2.507893e+06,2.507802e+06
